In [ ]:
from lib import *
import scipy.integrate as ode
from tqdm import tqdm

In [ ]:
N = 100
a, b = [0, 10]
dt, T = 0.02, 180

source_args = {'source_type': 'normal', 'source_mu': 6, 'source_std': 0.45}
theta_args = {'theta_value': 4, 'theta_moving': False, 'theta_speed': 0.00, 
              'theta_noise': False, 'theta_noise_type': 'normal', 'theta_noise_std': 0.025,"accumulative":False}
s_args = {'s_value': 1, 's_noise': True, 's_noise_type': 'normal', 's_noise_std': 0.25}

solver = model(N,a,b,dt,T)
solver.solve("solve_implicit", theta_args, s_args, source_args)

In [ ]:
solver.plot_2D(log_plot=False, nb_images=50)
solver.plot_params(pressure_selection=True)
#solver.plot_moments()

In [ ]:
def trapezoid(f,t0,t1): # numerical integration
    n = len(f)
    return ((t1-t0)/n) * (f[0]/2 + f[-1]/2 + np.sum(f[1:-1]))

#  Gaussian source

In [ ]:
N = 200
a, b = [0, 10]
dt, T = 0.02, 180
nbT = int(T/dt)

source_args = {'source_type': 'normal', 'source_mu': 6, 'source_std': 0.45}
theta_args = {'theta_value': 4, 'theta_moving': False, 'theta_speed': 0.00, 
              'theta_noise': False, 'theta_noise_type': 'normal', 'theta_noise_std': 0.025,"accumulative":False}
s_args = {'s_value': 1, 's_noise': True, 's_noise_type': 'normal', 's_noise_std': 0.25}

solver = model(N,a,b,dt,T)

simulations = 30

mu = np.zeros((nbT+1, simulations))
var = np.zeros_like(mu)
rho = np.zeros_like(mu)
FE = np.zeros_like(mu)
S = np.zeros_like(mu)

for i in tqdm(range(simulations)):
    solver.solve("solve_implicit", theta_args, s_args, source_args, disable_tqdm=True)
    mu[:,i] = solver.MU
    var[:,i] = solver.VAR
    rho[:,i] = solver.RHO
    FE[:,i] = solver.S * (solver.MU - solver.THETA)**2
    S[:, i] = solver.S

In [ ]:
dtt = np.linspace(0,T,nbT+1)
plt.figure(figsize=(26,5))
plt.subplot(1,4,1)
plt.title("Pressure selection")
plt.plot(dtt,np.mean(S,axis=-1), '-')
# plt.plot(dtt,np.ones(len(dtt)) * S_bar, 'r-')
#plt.plot(dtt,mu[:,-4:], '-')

plt.subplot(1,4,2)
plt.title("Var")
plt.plot(dtt,np.mean(var,axis=-1), '-')
plt.subplot(1,4,3)
plt.title("Rho")
plt.plot(dtt,np.mean(rho,axis=-1), '-')
plt.subplot(1,4,4)
plt.title("F")
plt.plot(dtt,np.mean(FE,axis = -1))
plt.show()


In [ ]:
N = 200
a, b = [0, 10]
dt, T = 0.02, 180
nbT = int(T/dt)

source_args = {'source_type': 'normal', 'source_mu': 5, 'source_std': 0.1}
theta_args = {'theta_value': 4, 'theta_moving': False, 'theta_speed': 0.00, 
              'theta_noise': True, 'theta_noise_type': 'normal', 'theta_noise_std': 0.05, "accumulative": False}
s_args = {'s_value': 1, 's_noise': False, 's_noise_type': 'normal', 's_noise_std': 0.01}

solver = model(N,a,b,dt,T)
solver.solve("solve_implicit", theta_args, s_args, source_args, disable_tqdm=False)


In [ ]:
t0, t1 = 100, 170
n_t0, n_t1 = int(t0/dt), int(t1/dt)

MU_bar = (1/(t1-t0)) * ode.trapz(solver.MU[n_t0:n_t1])*solver.dt
VAR_bar = (1/(t1-t0)) * ode.trapz(solver.VAR[n_t0:n_t1])*solver.dt
RHO_bar = (1/(t1-t0)) * ode.trapz(solver.RHO[n_t0:n_t1])*solver.dt
FE_bar = (1/(t1-t0)) * ode.trapz(solver.S[n_t0:n_t1]*(solver.MU[n_t0:n_t1]-solver.THETA[n_t0:n_t1])**2)*solver.dt
S_bar = (1/(t1-t0)) * ode.trapz(solver.S[n_t0:n_t1])*solver.dt

In [ ]:
dtt = np.linspace(0,T,nbT+1)
plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
plt.title("Phenotypic mean")
plt.plot(dtt,np.mean(mu, axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * MU_bar, 'r-')

plt.subplot(2,2,2)
plt.title("Variance")
plt.plot(dtt,np.mean(var,axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * VAR_bar, 'r-')

plt.subplot(2,2,3)
plt.title("Population size")
plt.plot(dtt,np.mean(rho,axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * RHO_bar, 'r-')

plt.subplot(2,2,4)
plt.title("Evolutionary load")
plt.plot(dtt,np.mean(FE,axis = -1))
plt.plot(dtt,np.ones(len(dtt)) * FE_bar, 'r-')

plt.show()

# Uniform source

In [ ]:
N = 200
a, b = [0, 10]
dt, T = 0.02, 180
nbT = int(T/dt)

source_args = {'source_type': 'normal', 'source_mu': 6, 'source_std': 0.45}
theta_args = {'theta_value': 4, 'theta_moving': False, 'theta_speed': 0.00, 
              'theta_noise': False, 'theta_noise_type': 'normal', 'theta_noise_std': 0.025,"accumulative":False}
s_args = {'s_value': 1, 's_noise': True, 's_noise_type': 'normal', 's_noise_std': 0.25}

solver = model(N,a,b,dt,T)
simulations = 30
mu = np.zeros((nbT+1, simulations))
var = np.copy(mu)
rho = np.copy(mu)
FE = np.copy(mu)
S = np.copy(mu)
for i in tqdm(range(simulations)):
    solver.solve("solve_implicit", theta_args, s_args, source_args, disable_tqdm=True)
    mu[:,i] = solver.MU
    var[:,i] = solver.VAR
    rho[:,i] = solver.RHO
    FE[:,i] = solver.S * (solver.MU - solver.THETA)**2
    S[:, i] = solver.S

In [ ]:
dtt = np.linspace(0,T,nbT+1)
plt.figure(figsize=(26,5))
plt.subplot(1,4,1)
plt.title("Pressure selection")
plt.plot(dtt,np.mean(S,axis=-1), '-')
# plt.plot(dtt,np.ones(len(dtt)) * S_bar, 'r-')
#plt.plot(dtt,mu[:,-4:], '-')

plt.subplot(1,4,2)
plt.title("Var")
plt.plot(dtt,np.mean(var,axis=-1), '-')
plt.subplot(1,4,3)
plt.title("Rho")
plt.plot(dtt,np.mean(rho,axis=-1), '-')
plt.subplot(1,4,4)
plt.title("F")
plt.plot(dtt,np.mean(FE,axis = -1))
plt.show()

In [ ]:
N = 200
a, b = [0, 10]
dt, T = 0.02, 180
nbT = int(T/dt)

source_args = {'source_type': 'normal', 'source_mu': 6, 'source_std': 0.45}
theta_args = {'theta_value': 4, 'theta_moving': False, 'theta_speed': 0.00, 
              'theta_noise': False, 'theta_noise_type': 'normal', 'theta_noise_std': 0.025,"accumulative":False}
s_args = {'s_value': 1, 's_noise': True, 's_noise_type': 'normal', 's_noise_std': 0.25}


solver = model(N,a,b,dt,T)
solver.solve("solve_implicit", theta_args, s_args, source_args)


In [ ]:
t0 = 100
t1 = 180
n_t0 = int(t0/dt)
n_t1 = int(t1/dt)
MU_bar = (1/(t1-t0)) * ode.trapz(solver.MU[n_t0:n_t1])*solver.dt
VAR_bar = (1/(t1-t0)) * ode.trapz(solver.VAR[n_t0:n_t1])*solver.dt
RHO_bar = (1/(t1-t0)) * ode.trapz(solver.RHO[n_t0:n_t1])*solver.dt
FE_bar = (1/(t1-t0)) * ode.trapz(solver.S[n_t0:n_t1]*(solver.MU[n_t0:n_t1]-solver.THETA[n_t0:n_t1])**2)*solver.dt
S_bar = (1/(t1-t0)) * ode.trapz(solver.S[n_t0:n_t1])*solver.dt

In [ ]:
dtt = np.linspace(0,T,nbT+1)
plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
plt.title("Phenotypic mean")
plt.plot(dtt,np.mean(mu, axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * MU_bar, 'r-')
# plt.ylim(0.75,1.25)

plt.subplot(2,2,2)
plt.title("Variance")
plt.plot(dtt,np.mean(var,axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * VAR_bar, 'r-')
# plt.ylim(0,1)

plt.subplot(2,2,3)
plt.title("Population size")
plt.plot(dtt,np.mean(rho,axis=-1), '-')
plt.plot(dtt,np.ones(len(dtt)) * RHO_bar, 'r-')
# plt.ylim(0,1)

plt.subplot(2,2,4)
plt.title("Evolutionary load")
plt.plot(dtt,np.mean(FE,axis = -1))
plt.plot(dtt,np.ones(len(dtt)) * FE_bar, 'r-')
# plt.ylim(0,0.01)

plt.show()


# oscillations as a function of number of realisations

In [ ]:
bfff = False
if bfff:   
    sims = np.arange(1,11)*20
    N = 200
    a, b = [0, 10]
    dt, T = 0.02, 100
    nbT = int(T/dt)

    source_args = {'source_type': 'uniform', 'source_mu': 5, 'source_std': 0.1}
    theta_args = {'theta_value': 4.5, 'theta_moving': False, 'theta_speed': 0.00, 
                  'theta_noise': True, 'theta_noise_type': 'normal', 'theta_noise_std': 0.05, "accumulative" : False}
    s_args = {'s_value': 1, 's_noise': False, 's_noise_type': 'normal', 's_noise_std': 0.01}

    solver = model(N,a,b,dt,T)
    mu_sims = np.zeros((nbT+1, len(sims)))
    var_sims = np.copy(mu_sims)
    rho_sims = np.copy(mu_sims)
    FE_sims = np.copy(mu_sims)
    for j,s in enumerate(sims):
        mu = np.zeros((nbT+1, s))
        var = np.copy(mu)
        rho = np.copy(mu)
        FE = np.copy(mu)
        for i in tqdm(range(s)):

            solver.solve("solve_implicit", theta_args, s_args, source_args,show_tqdm = False)
            mu[:,i] = solver.MU
            var[:,i] = solver.VAR
            rho[:,i] = solver.RHO
            FE[:,i] = solver.S*(solver.MU - solver.THETA)**2

        mu_sims[:,j] = np.mean(mu, axis = -1)
        var_sims[:,j] = np.mean(var, axis = -1)
        rho_sims[:,j] = np.mean(rho, axis = -1)
        FE_sims[:,j] = np.mean(FE,axis = -1)

In [ ]:
import pandas as pd

In [ ]:
#pd.DataFrame(mu_sims).to_pickle("data/mu.pkl")
#pd.DataFrame(var_sims).to_pickle("data/var.pkl")
#pd.DataFrame(rho_sims).to_pickle("data/rho.pkl")
#pd.DataFrame(FE_sims).to_pickle("data/FE.pkl")

In [ ]:
FE = pd.read_pickle("data/FE.pkl")
sims = np.arange(1,11)*20


In [ ]:
dtt = np.linspace(0,T,nbT+1)
plt.figure(figsize = (14,10))
for i in range(len(FE.columns)):
    plt.plot(dtt, FE[i].values, label = str(sims[i]))
plt.legend()
plt.xlim(0.1, 100)
plt.ylim(-0.0025, 0.008)
plt.title("Fardeau evolutif en fonction de nombre de realisations")
plt.plot(dtt, np.ones(len(dtt))*FE_bar, label =  'mean')
plt.show()